# Evaluation Recall Analysis

One thing that the current preview experiments demonstrated (50_IP, 50_L2) it's that most of the collections evlauated in the top 50 are retrieving 0  
The goal with this notebook it's investigate possible patterns for the recurrency of this behavior  
In order to mitigate that we tried an approach where the default instruction prompt tried to incetivate the answer generation  
Here we will anlyse the following questions:
- The indexation retrieval impact in the massive lack of answers?
- The mean distance of the retrieval correlates with the lack of answer?
- The differentiation in the prompt was effective to increase the recall?

# Imports

In [1]:
import os
import random
import numpy as np
import torch

import json
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from dmcr.evaluators.Rouge_L_evaluator import Rouge_L_evaluator

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
seed = 42

torch.backends.cudnn.enabled = False
# NumPy
np.random.seed(seed)
random.seed(seed)
# PyTorch
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False



if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/caio.rhoden/miniconda3/envs/nq/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of GPUs available: 1
GPU 0: NVIDIA RTX A5000


In [ ]:
L2_PATH = "preview_50_L2"
IP_PATH = "preview_50_IP"
COSINE_PATH = "new_prompt_preview_50_cosine"
K8_PATH = "preview_50_L2_k8"


## The indexation retrieval impact in the massive lack of answers?

In order to this we will calculate the number of non-zero collectionss samples  
The maximun of samples of 100k the setup of comparison it's the 4 documents in the context using Llama-3.2-8B-Instruct  


In [9]:
## Load collections by index
train_collections = []
test_collections = []
for collection_path in [L2_PATH, IP_PATH, K8_PATH]:
    df_train = pl.read_ipc(f"{collection_path}/datamodels/collections/train/{collection_path}_train_collection.feather").with_columns(pl.lit(f"{collection_path}").alias("collection_path"))
    df_test = pl.read_ipc(f"{collection_path}/datamodels/collections/test/{collection_path}_test_collection.feather").with_columns(pl.lit(f"{collection_path}").alias("collection_path"))
    train_collections.append(df_train)
    test_collections.append(df_test)

train_collections = pl.concat(train_collections)
test_collections = pl.concat(test_collections)

FileNotFoundError: No such file or directory (os error 2): preview_50_L2_k8/datamodels/collections/train/preview_50_L2_k8_train_collection.feather

In [ ]:
_temp

collection_idx,test_idx,input,evaluation,collection_path
i64,i64,"array[i64, 100]",f64,str
1000,0,"[0, 0, … 0]",0.0,"""preview_50_L2"""
1000,1,"[0, 0, … 0]",0.0,"""preview_50_L2"""
1000,2,"[0, 0, … 0]",0.0,"""preview_50_L2"""
1000,3,"[0, 0, … 0]",0.0,"""preview_50_L2"""
1000,4,"[0, 0, … 0]",0.0,"""preview_50_L2"""
…,…,…,…,…
999,45,"[0, 0, … 0]",0.0,"""preview_50_IP"""
999,46,"[0, 0, … 0]",0.0,"""preview_50_IP"""
999,47,"[0, 0, … 0]",0.285714,"""preview_50_IP"""
